In [1]:
%%configure -f
{
 "executorCores": 4, 
 "executorMemory": "47696M", 
 "conf": {"spark.default.parallelism": 1000,
          "spark.sql.shuffle.partitions": 1000,
          "spark.task.cpus": 1
         }
}

In [2]:
word2id_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/common"
data_path = "s3://oaknorth-ml-dev-eu-west-1/company2vec/data_desc_only"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1574682469228_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from gensim.utils import lemmatize, tokenize
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from langdetect import detect
import string
from itertools import islice
import numpy as np
import smart_open
from collections import Counter
import math
import spacy

from pyspark.ml.feature import (HashingTF,
                                IDF,
                                Tokenizer,
                                StopWordsRemover,
                                CountVectorizer,
                                StringIndexer,
                                OneHotEncoderEstimator,
                                VectorAssembler,
                                VectorSizeHint,
                                StandardScaler,
                                PCA
                               )
from pyspark.ml import Pipeline, Transformer

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

from pyspark.ml.linalg import Vectors, VectorUDT

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# spark = (
#     SparkSession.builder
#      .master("local[*]")
#      .appName("SparkTest")
#      .getOrCreate()
#      )

In [9]:
companies_raw = spark.read.load("s3://ai-data-lake-dev-eu-west-1/business/company_data_denormalized")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
target_doc = (companies_raw.filter(F.col("entity_id") == '9455d310-201a-5545-b073-fecbc49cbd94')
                  .first()
                 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
target_doc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False
is_english_udf = F.udf(is_english, T.BooleanType())

def tokenize_text(text, remove_sws=False):
    if not text:
        return []
    if remove_sws:
        text = remove_stopwords(text)
    text = tokenize(text, lowercase=True)
    
    return text
tokenize_text_udf = F.udf(tokenize_text, T.ArrayType(T.StringType()))

p = PorterStemmer()
def lemmatize_text(text):
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in text]
    words = [p.stem(word) for word in words]
    return list(
        filter(lambda word: word not in string.punctuation and word.isalpha() and len(word) > 1, words)
    )
lemmatize_text_udf = F.udf(lemmatize_text, T.ArrayType(T.StringType()))

def sparse_bow(*args):
    ret = []
    for el in args:
        if not el:
            continue
        ret.extend(el)
    return dict(Counter(ret))
sparse_bow_udf = F.udf(sparse_bow, T.MapType(T.StringType(), T.IntegerType()))

join_text_udf = F.udf(lambda words: " ".join(words), T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
duplicate_ids = [row.entity_id for row in
                 companies_raw
                 .groupBy("entity_id")
                 .agg(F.count("*").alias("count"))
                 .filter(F.col("count") > 1)
                 .collect()
                 ]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
(companies_raw
 .filter(~F.col("entity_id").isin(duplicate_ids))
 .filter((F.col("business_description").isNotNull() &
                     (F.length("business_description") > 0))
                    )
).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

900449

In [8]:
companies = (companies_raw
             .filter(~F.col("entity_id").isin(duplicate_ids))
             .filter((F.col("business_description").isNotNull() &
                     (F.length("business_description") > 0))
                    )
             .withColumn("bow_description_lemmatized",
                         sparse_bow_udf(lemmatize_text_udf(tokenize_text_udf("business_description", F.lit(True))))
                        )
             .withColumn("bow_description_not_lemmatized",
                         sparse_bow_udf(tokenize_text_udf("business_description"))
                        )
             .withColumn("ebitda_marg_calc", F.col("ebitda")/F.col("total_revenue"))
             .filter(F.size("bow_description_lemmatized") > 0)
             .repartition(1000)
).cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
companies.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

900419

In [9]:
(companies
 .write
 .parquet(f"{data_path}/enhanced_companies_es", mode="overwrite")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
tst_companies_enhanced = spark.read.load(f"{data_path}/enhanced_companies_es")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
tst_companies_enhanced.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3464992

In [10]:
num_docs = companies.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def save_words(description_column, min_df=0.00001):
    min_df = num_docs*0.00001
    
    words_df = {row.key: row.n_docs for row in
            companies.select(F.explode(description_column))
                     .groupBy("key")
                     .agg(F.count("*").alias("n_docs"))
                     .filter(F.col("n_docs") > min_df)
                     .collect()
            }
    num_words = len(words_df)
    words_idf = {k: math.log((num_docs+1)/(v+1)) for k,v in words_df.items()}
    idx = 0
    id2word = {}
    word2id = {}
    for word in sorted(words_idf):
        id2word[idx] = word
        word2id[word] = idx
        idx += 1
    
    print(f"Saving words to {word2id_path}/{description_column}")
    with smart_open.open(f"{word2id_path}/{description_column}/words_idf.csv", "w") as f:
        for word,idf in sorted(words_idf.items()):
            f.write(f"{word},{idf}\n")
    with smart_open.open(f"{word2id_path}/{description_column}/word2id.csv", "w") as f:
        for word,idd in sorted(word2id.items()):
            f.write(f"{word},{idd}\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
save_words("bow_description_lemmatized")
save_words("bow_description_not_lemmatized")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Saving words to s3://oaknorth-ml-dev-eu-west-1/company2vec/common/bow_description_lemmatized
Saving words to s3://oaknorth-ml-dev-eu-west-1/company2vec/common/bow_description_not_lemmatized

In [13]:
def extract_features(feature_column):
    words_idf = {}
    with smart_open.open(f"{word2id_path}/{feature_column}/words_idf.csv", "r") as f:
        for line in f:
            word,idf = line.strip().split(",")
            words_idf[word] = float(idf)

    word2id = {}
    with smart_open.open(f"{word2id_path}/{feature_column}/word2id.csv", "r") as f:
        for line in f:
            word,idd = line.strip().split(",")
            word2id[word] = int(idd)
    num_words = len(word2id)
    
    def words_tfidf(bow):
        dct = {word2id[k]: math.log(v+1)*words_idf[k] for k,v in bow.items() if k in word2id}
        return Vectors.sparse(num_words, dct)
    words_tfidf_udf = F.udf(words_tfidf, VectorUDT())
    
    companies_tfidf = (companies
                   .withColumn("bow_tfidf", words_tfidf_udf(feature_column))
                   .cache()
                   )
    
    vectorizer = VectorAssembler(inputCols=["bow_tfidf",
                                       ],
                             outputCol="features",
                             handleInvalid = "skip"
                            )

    pipeline = Pipeline(stages=[vectorizer])

    pipeline_fit = pipeline.fit(companies_tfidf)
    processed_companies = pipeline_fit.transform(companies_tfidf).repartition(1000)
    
    get_size    = F.udf(lambda vec: vec.size, T.IntegerType())
    get_indices = F.udf(lambda vec: [int(el) for el in vec.indices], T.ArrayType(T.IntegerType()))
    get_values  = F.udf(lambda vec: [float(el) for el in vec.values], T.ArrayType(T.DoubleType()))
    
    print(f"Saving data to {data_path}/raw_company_features_{feature_column}")
    
    (processed_companies.select("entity_id",
                            get_size("features").alias("size"),
                            get_indices("features").alias("feature_indices"),
                            get_values("features").alias("feature_values"),
                            feature_column
                           )
         .repartition(32)
         .write
         .parquet(f"{data_path}/raw_company_features_{feature_column}", mode="overwrite")
        )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
extract_features("bow_description_lemmatized")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Saving data to s3://oaknorth-ml-dev-eu-west-1/company2vec/data_desc_only/raw_company_features_bow_description_lemmatized

In [15]:
extract_features("bow_description_not_lemmatized")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Saving data to s3://oaknorth-ml-dev-eu-west-1/company2vec/data_desc_only/raw_company_features_bow_description_not_lemmatized

In [52]:
def make_checker_udf(categories):    
    def check_onehot(column, vector):
        if not column:
            return not np.any(vector)
        i=-1
        for ii,cat in enumerate(categories):
            if cat == column:
                i=ii
                break
        return bool(vector[i] == 1.0)
    return F.udf(check_onehot, T.BooleanType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
companies_df = spark.read.load(f"{data_path}/raw_company_features_bow_description_lemmatized")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
target_doc = (companies_df.filter(F.col("entity_id") ==  'da166d49-acbc-54c5-9ab4-d72a143920f6')
                  .first()
                 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
target_doc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
companies_df.show(20, False)

+------------------------------------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------